Importing Various libraries needed.
Floium for displaying map
geopy for getting location data for a specific location
ssl, certifi to ignore ssl certificate Verification 

Problem Statement
Food is an integral part of the culture of a place. When we visit a place it might be difficult to select a restaurant as we expect different things like ambiance, budget etc. This project uses FourSquare Api to collect restaurants near CP ,Delhi, India. Is used Folium to plot on a map, geolocator to get latitudes and longitude of a neighbourhood. It uses sklearn to make a model which takes reviews and likes of a place and can recommend a place based on your input of string.

In [1]:
import requests
import pandas as pd
import folium
import numpy as np
import geopy.geocoders
from geopy.geocoders import Nominatim
import certifi
import time
import ssl
import csv

I am collecting data of a particular location in Delhi, India. Using address as Connaught Place, Delhi, i will get the location of Connaught Place, Delhi when passed through geolocator.
SSL verifivation is necessary to ignore as it is giving an error on my machine.

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
geopy.geocoders.options.default_ssl_context = ctx
geolocator = Nominatim()
address = 'Berlin'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/Users/lonetitan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


In [24]:
longitude

13.3888599

GetNearbyVenues takes a list entry of name, latitudes, longitudes and default parameter of 500 m
I have declared Client_ID, Client_Secret and version in the beginning.I have a list venue_list to store all the venues nearby the Connaught Place, Delhi. Condition used in api used is selection : food to get food places address, latitude, longitude, name, category and id.
Limit is set to 100 to get max 100 places near the location.

In [25]:
def getNearbyVenues(name, latitudes, longitudes, radius = 500):
    #Pass in list of names, latitude, longitude
    CLIENT_ID = 'CFRMV25U5F3IE54QYOQMNO1DUPBYFAIIY3O21O4SWIBOPJKY' # your Foursquare ID
    CLIENT_SECRET = 'MFU5O5EXRPA3Q2UETJKWBU1KI3DV3WDYRSQ1ZNXDJAJVLW1E' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    venues_list=[]
    print(name)
    limit = 100
    radius = 500
    temp_latitude = latitudes
    temp_longitude = longitudes
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=food'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        temp_latitude, 
        temp_longitude, 
        radius, 
        limit)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    venues_list.append([(
        name, 
        latitude, 
        longitude, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name'],
        v['venue']['id']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
            'Neighbourhood Latitude', 
            'Neighbourhood Longitude', 
            'Venue',
            'Venue Latitude', 
            'Venue Longitude', 
            'Venue Category',
            'Venue Id']
    return(nearby_venues)

GetLikes function takes input of names list and their venue id.
Four Square API is called to get the likes of the venue whose venue id is passed.
This information is stored in like_list which is the passed.

In [26]:
import json
def getlikes(names,venue_id, radius = 500):
    CLIENT_ID = 'CFRMV25U5F3IE54QYOQMNO1DUPBYFAIIY3O21O4SWIBOPJKY' # your Foursquare ID
    CLIENT_SECRET = 'MFU5O5EXRPA3Q2UETJKWBU1KI3DV3WDYRSQ1ZNXDJAJVLW1E' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    like_list=[]
    for name,venid in zip(names, venue_id):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}'.format(
        venid,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION)
        like_result = requests.get(url).json()['response']
        like_list.append(like_result['likes']['count'])
    return like_list

Finally calling the function getnearbyvenues on "Connaught Place, Delhi and storing the result in dataframe df

In [27]:
df = getNearbyVenues("Berlin",latitude,longitude )

Berlin


THis function takes list of venue id and extracts the tip about the venue along with user id. Since calls are limited I am using two accounts for fetching the data. When calls excede 45, it will switch to different ID

In [28]:
def getReviews(venueId):
    count = 0;
    review = []
    userid = []
    for(venId) in zip(venueId):
        CLIENT_ID = 'CFRMV25U5F3IE54QYOQMNO1DUPBYFAIIY3O21O4SWIBOPJKY' # your Foursquare ID
        CLIENT_SECRET = 'MFU5O5EXRPA3Q2UETJKWBU1KI3DV3WDYRSQ1ZNXDJAJVLW1E' # your Foursquare Secret
        VERSION = '20180605' # Foursquare API version
        
        url = 'https://api.foursquare.com/v2/venues/{}/tips?&client_id={}&client_secret={}&v={}'.format(
            venId[0],
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION )
        results = requests.get(url).json()
        try:
            review.append(results["response"]['tips']['items'][0]['text'])
            userid.append(results["response"]['tips']['items'][0]['id'])
        
        except:
            review.append("Bad")
            userid.append('1a21ab1c498e411929349f40')
        
    return review,userid

Finally storing the data from getReviews into the dataframe with col names 'Reviews', 'User Id'

In [29]:
df['Reviews'],df['User Id'] = getReviews(df['Venue Id'])

Finally storing the data of likes in df with col name 'LIKES'

In [30]:
df['Likes'] = getlikes(df['Venue'], df['Venue Id'])

Cookies Cream
Ishin
Crackers
Café Einstein
Gendarmerie
Bocca di Bacco
Café Ursprung
Borchardt
Nante-Eck
Flamingo Fresh Food Bar
Swing Kitchen
La Banca
Mishba
Restaurant Nolle
Augustiner am Gendarmenmarkt
Haferkater
Lutter & Wegner Gendarmenmarkt
Ständige Vertretung
Restaurant Coelln
Austernbank | Gendarmerie Berlin
Relish Restaurant & Bar
Block House
TeX-Mex Cantina
Zen Asian Kitchen
Brechts
ZDF Cafe
Peter Pane
Restaurant 1687
Gaffel Haus Berlin
Beef Grill Club By Hasir
Ristorante Il Punto
Dolcini
Prime Kebap
Via Nova II
Mosaik Kantine
Bistro am Gendarmenmarkt
Charlotte & Fritz
LE CROBAG
Deponie Nr. 3
Cantina
Ristorante DaVinci
Soya Cosplay
Refugium
Shan's Bistro
Berliner Kindl
Tim's Bakery and Deli
Erdinger Am Gendarmenmarkt
Café Konzerthaus
Mare e Monti
Restaurant Cum Laude
Fish Point
Cafeteria HU Berlin
Cafe Quartier 206
Amici am Gendarmenmarkt
Café Chagall


Final data frame check. After this all the collection of data has been completed.

In [31]:
df

Neighbourhood  Neighbourhood Latitude  Neighbourhood Longitude  \
0         Berlin               52.517037                 13.38886   
1         Berlin               52.517037                 13.38886   
2         Berlin               52.517037                 13.38886   
3         Berlin               52.517037                 13.38886   
4         Berlin               52.517037                 13.38886   
5         Berlin               52.517037                 13.38886   
6         Berlin               52.517037                 13.38886   
7         Berlin               52.517037                 13.38886   
8         Berlin               52.517037                 13.38886   
9         Berlin               52.517037                 13.38886   
10        Berlin               52.517037                 13.38886   
11        Berlin               52.517037                 13.38886   
12        Berlin               52.517037                 13.38886   
13        Berlin               52.517037                 13.38886   
14        Berlin               52.517037                 13.38886   
15        Berlin               52.517037                 13.38886   
16        Berlin               52.517037                 13.38886   
17        Berlin               52.517037                 13.38886   
18        Berlin               52.517037                 13.38886   
19        Berlin               52.517037                 13.38886   
20        Berlin               52.517037                 13.38886   
21        Berlin               52.517037                 13.38886   
22        Berlin               52.517037                 13.38886   
23        Berlin               52.517037                 13.38886   
24        Berlin               52.517037                 13.38886   
25        Berlin               52.517037                 13.38886   
26        Berlin               52.517037                 13.38886   
27        Berlin               52.517037                 13.38886   
28        Berlin               52.517037                 13.38886   
29        Berlin               52.517037                 13.38886   
30        Berlin               52.517037                 13.38886   
31        Berlin               52.517037                 13.38886   
32        Berlin               52.517037                 13.38886   
33        Berlin               52.517037                 13.38886   
34        Berlin               52.517037                 13.38886   
35        Berlin               52.517037                 13.38886   
36        Berlin               52.517037                 13.38886   
37        Berlin               52.517037                 13.38886   
38        Berlin               52.517037                 13.38886   
39        Berlin               52.517037                 13.38886   
40        Berlin               52.517037                 13.38886   
41        Berlin               52.517037                 13.38886   
42        Berlin               52.517037                 13.38886   
43        Berlin               52.517037                 13.38886   
44        Berlin               52.517037                 13.38886   
45        Berlin               52.517037                 13.38886   
46        Berlin               52.517037                 13.38886   
47        Berlin               52.517037                 13.38886   
48        Berlin               52.517037                 13.38886   
49        Berlin               52.517037                 13.38886   
50        Berlin               52.517037                 13.38886   
51        Berlin               52.517037                 13.38886   
52        Berlin               52.517037                 13.38886   
53        Berlin               52.517037                 13.38886   
54        Berlin               52.517037                 13.38886   

                               Venue  Venue Latitude  Venue Longitude  \
0                      Cookies Cream       52.516569        13.38

Writing the data in a csv file so that it can be used later without exhausting all the API calls
Write in made which writes all the entries of df into the file.

In [32]:
with open ("DelhiRestaurants.csv", 'a') as f:
    writer = csv.writer(f, delimiter = ",")
#     col = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbour Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category', 'Venue Id', 'Reviews','User Id', 'Likes']
#     writer.writerow(col)
    for data in range(0,df.shape[0]):
        writer.writerow(df.iloc[data, :])
df.head()

Neighbourhood  Neighbourhood Latitude  Neighbourhood Longitude  \
0        Berlin               52.517037                 13.38886   
1        Berlin               52.517037                 13.38886   
2        Berlin               52.517037                 13.38886   
3        Berlin               52.517037                 13.38886   
4        Berlin               52.517037                 13.38886   

           Venue  Venue Latitude  Venue Longitude  \
0  Cookies Cream       52.516569        13.388008   
1          Ishin       52.517986        13.386714   
2       Crackers       52.516495        13.388118   
3  Café Einstein       52.517109        13.385822   
4    Gendarmerie       52.515719        13.390940   

                  Venue Category                  Venue Id  \
0  Vegetarian / Vegan Restaurant  4adf61aef964a520177a21e3   
1               Sushi Restaurant  4ba21485f964a5205fda37e3   
2                     Restaurant  54610138498e2db343de55ab   
3                           Café  4afa6086f964a520c91722e3   
4              German Restaurant  4b0a99d8f964a520822523e3   

                                             Reviews  \
0  If you find yourself in an alley place that lo...   
1  Great Lunchspot in Mitte. Often crowded: takes...   
2  Vorab einen Cocktail an der Bar (mit Roter Bet...   
3  Very classical café in the center of Berlin. O...   
4  Great venue with high ceilings. Customers are ...   

                    User Id  Likes  
0  57bab4e8cd10c2b6bccf60eb    371  
1  590349dce97dfb22af890cb9    436  
2  5795238d498e8f343a0ae75b    171  
3  586689fd65ca0133fd3ee77b    760  
4  5b8db6b0ba57b4002ce02092    147

For the final show I am importing sklearn to vectorize the features in reviews and importing the nltk(Natural Languare Tool Kit) to clean the reviews.

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer

This algorithm removes all emoji, punctuation, nouns and other words which are for the language sake and will not provide any valid token.

In [4]:
import string
import re
def clean_text(reviews):
    res = []
    for text in reviews :
        text = text.translate(string.punctuation)

        ## Convert words to lower case and split them
        text = text.lower().split()

        ## Remove stop words
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops and len(w) >= 3]

        text = " ".join(text)

        # Clean the text
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"\/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"\-", " - ", text)
        text = re.sub(r"\=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)    
        res.append(text)
    return res

In [5]:
df = pd.read_csv("DelhiRestaurants.csv")

Cleaning the data and rewriting the data in reviews and copying the original comments in teh data frame in Reviews_ori

In [6]:
df['Reviews_ori'] = df['Reviews']
df['Reviews'] = clean_text(df['Reviews'])

Now making separate list frame for user id and venue id

In [7]:
user_df = df[['User Id', 'Reviews']]
venue_df = df[['Venue Id', 'Reviews']]

in user_df I am making a paragraph of all the comments given out by a user and joining them.
This is done with venue_df. All the reviews written about venue_id is made into a paragraph and then added to the corresponding venue_id

In [8]:
user_df = user_df.groupby('User Id').agg({'Reviews':' '.join})
venue_df = venue_df.groupby('Venue Id').agg({'Reviews':' '.join})

Vectorizing all the features of the reviews given out by the user, max features allowed 1000, to make a judgement that what "good" means. Similar process is gone with venues and the check is likes. Is is assumed that restaurant will be consistant with the review and hence words or features can be assigned a meaning.

In [9]:
#User Id vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=1000)
userid_vectors = userid_vectorizer.fit_transform(user_df['Reviews'])
userid_vectors.shape

(472, 1000)

In [10]:
#Venue Id vectorizer
venue_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=1000)
venue_vectors = venue_vectorizer.fit_transform(venue_df['Reviews'])
venue_vectors.shape

(544, 1000)

Making two data frames with userid, and features for both user and venue and naming that P for user and Q for Venue

In [11]:
P = pd.DataFrame(userid_vectors.toarray(), index=user_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(venue_vectors.toarray(), index=venue_df.index, columns=venue_vectorizer.get_feature_names())


Let us see P

In [12]:
P.head()

!    +         -    1   10  100   11   12   14  \
User Id                                                                      
1a21ab1c498e411929349f40  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
4d581d8392326ea81b3a62c0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
4dde761c81dc93c03cd8921f  0.0  0.0  0.160786  0.0  0.0  0.0  0.0  0.0  0.0   
4e048c7081dc9d212d105670  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
4e29c9f718a80bb05845f810  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   

                            2  ...  would  wrap  wraps  wrong  years  yet  \
User Id                        ...                                          
1a21ab1c498e411929349f40  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   
4d581d8392326ea81b3a62c0  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   
4dde761c81dc93c03cd8921f  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   
4e048c7081dc9d212d105670  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   
4e29c9f718a80bb05845f810  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   

                          york  you  yummy  zum  
User Id                                          
1a21ab1c498e411929349f40   0.0  0.0    0.0  0.0  
4d581d8392326ea81b3a62c0   0.0  0.0    0.0  0.0  
4dde761c81dc93c03cd8921f   0.0  0.0    0.0  0.0  
4e048c7081dc9d212d105670   0.0  0.0    0.0  0.0  
4e29c9f718a80bb05845f810   0.0  0.0    0.0  0.0  

[5 rows x 1000 columns]

Let us see Q

In [13]:
Q.head()

!    +         -    1   10       100   11  \
Venue Id                                                                     
3fd66200f964a52052ec1ee3  0.000000  0.0  0.000000  0.0  0.0  0.000000  0.0   
3fd66200f964a520f4e41ee3  0.099600  0.0  0.140666  0.0  0.0  0.272089  0.0   
40b28c80f964a52098f71ee3  0.000000  0.0  0.000000  0.0  0.0  0.000000  0.0   
40b28c80f964a5209ff71ee3  0.298188  0.0  0.000000  0.0  0.0  0.000000  0.0   
43dc103ef964a520992e1fe3  0.000000  0.0  0.000000  0.0  0.0  0.000000  0.0   

                           12   14    2  ...  would  wrap  wraps  wrong  \
Venue Id                                 ...                              
3fd66200f964a52052ec1ee3  0.0  0.0  0.0  ...    0.0   0.0    0.0    0.0   
3fd66200f964a520f4e41ee3  0.0  0.0  0.0  ...    0.0   0.0    0.0    0.0   
40b28c80f964a52098f71ee3  0.0  0.0  0.0  ...    0.0   0.0    0.0    0.0   
40b28c80f964a5209ff71ee3  0.0  0.0  0.0  ...    0.0   0.0    0.0    0.0   
43dc103ef964a520992e1fe3  0.0  0.0  0.0  ...    0.0   0.0    0.0    0.0   

                          years       yet  york  you  yummy  zum  
Venue Id                                                          
3fd66200f964a52052ec1ee3    0.0  0.000000   0.0  0.0    0.0  0.0  
3fd66200f964a520f4e41ee3    0.0  0.000000   0.0  0.0    0.0  0.0  
40b28c80f964a52098f71ee3    0.0  0.000000   0.0  0.0    0.0  0.0  
40b28c80f964a5209ff71ee3    0.0  0.000000   0.0  0.0    0.0  0.0  
43dc103ef964a520992e1fe3    0.0  0.307739   0.0  0.0    0.0  0.0  

[5 rows x 1000 columns]

Making a matrix with rows as row id and columns as venue id. All the corresponding values whihc are common in both are filles, NaN otherwise. It is expected that a lot of entries will be NaN as not all users have tried each and every restaurant in the city. So not need to freek out.

In [14]:
userid_rating_matrix = pd.pivot_table(df, values='Likes', index=['User Id'], columns=['Venue Id'])
userid_rating_matrix.shape

(472, 544)

In [15]:
userid_rating_matrix

Venue Id                  3fd66200f964a52052ec1ee3  3fd66200f964a520f4e41ee3  \
User Id                                                                        
1a21ab1c498e411929349f40                     202.0                       NaN   
4d581d8392326ea81b3a62c0                       NaN                       NaN   
4dde761c81dc93c03cd8921f                       NaN                       NaN   
4e048c7081dc9d212d105670                       NaN                       NaN   
4e29c9f718a80bb05845f810                       NaN                       NaN   
...                                            ...                       ...   
5e36136b4d5f1e00067c4039                       NaN                       NaN   
5e3e0674decf770007e65605                       NaN                       NaN   
5e52bc2283d3740006c51a6b                       NaN                       NaN   
5e52bff79b9e0c0007eba9bb                       NaN                       NaN   
5e662cde20853a0006ff7cba                       NaN                       NaN   

Venue Id                  40b28c80f964a52098f71ee3  40b28c80f964a5209ff71ee3  \
User Id                                                                        
1a21ab1c498e411929349f40                      17.0                       NaN   
4d581d8392326ea81b3a62c0                       NaN                       NaN   
4dde761c81dc93c03cd8921f                       NaN                       NaN   
4e048c7081dc9d212d105670                       NaN                       NaN   
4e29c9f718a80bb05845f810                       NaN                       NaN   
...                                            ...                       ...   
5e36136b4d5f1e00067c4039                       NaN                       NaN   
5e3e0674decf770007e65605                       NaN                       NaN   
5e52bc2283d3740006c51a6b                       NaN                       NaN   
5e52bff79b9e0c0007eba9bb                       NaN                       NaN   
5e662cde20853a0006ff7cba                       NaN                       NaN   

Venue Id                  43dc103ef964a520992e1fe3  43eb7d31f964a520392f1fe3  \
User Id                                                                        
1a21ab1c498e411929349f40                       NaN                       NaN   
4d581d8392326ea81b3a62c0                       NaN                       NaN   
4dde761c81dc93c03cd8921f                       NaN                       NaN   
4e048c7081dc9d212d105670                       NaN                       NaN   
4e29c9f718a80bb05845f810                       NaN                       NaN   
...                                            ...                       ...   
5e36136b4d5f1e00067c4039                       NaN                       NaN   
5e3e0674decf770007e65605                       NaN                       NaN   
5e52bc2283d3740006c51a6b                       NaN                       NaN   
5e52bff79b9e0c0007eba9bb                       NaN                       NaN   
5e662cde20853a0006ff7cba                       NaN                       NaN   

Venue Id                  45a152ecf964a520e0401fe3  45bc5a54f964a520f9411fe3  \
User Id                                                                        
1a21ab1c498e411929349f40                       NaN                       NaN   
4d581d8392326ea81b3a62c0                       NaN                       NaN   
4dde761c81dc93c03cd8921f                       NaN                       NaN   
4e048c7081dc9d212d105670                       NaN                       NaN   
4e29c9f718a80bb05845f810                       NaN                       NaN   
...                                            ...                       ...   
5e36136b4d5f1e00067c4039                       NaN                       NaN   
5e3e0674decf770007e65605                       NaN                       NaN   
5e52bc2283d3740006c51a6b                       NaN                       NaN 

The Objective of Matrix Factorization is to decompose each user rating into a User-Factor Vector and A Product-Vector. As we have too many NaN’s in our user item rating matrix hence we only solve the ratings which are available i.e. each rating has to be decomposed into 2 vectors.

In [16]:
def matrix_factorization(R,P,Q,steps=100,gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q


Call the matrix_factorization method with all the hyper parameter.

In [17]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=100, gamma=0.001,lamda=0.02)

CPU times: user 8min 20s, sys: 478 ms, total: 8min 21s
Wall time: 8min 21s


In [29]:
P.head()

!    +         -    1   10  100   11   12   14  \
User Id                                                                      
1a21ab1c498e411929349f40  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
4d581d8392326ea81b3a62c0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
4dde761c81dc93c03cd8921f  0.0  0.0  0.423933  0.0  0.0  0.0  0.0  0.0  0.0   
4e048c7081dc9d212d105670  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   
4e29c9f718a80bb05845f810  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0   

                            2  ...  would  wrap  wraps  wrong  years  yet  \
User Id                        ...                                          
1a21ab1c498e411929349f40  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   
4d581d8392326ea81b3a62c0  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   
4dde761c81dc93c03cd8921f  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   
4e048c7081dc9d212d105670  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   
4e29c9f718a80bb05845f810  0.0  ...    0.0   0.0    0.0    0.0    0.0  0.0   

                          york  you  yummy  zum  
User Id                                          
1a21ab1c498e411929349f40   0.0  0.0    0.0  0.0  
4d581d8392326ea81b3a62c0   0.0  0.0    0.0  0.0  
4dde761c81dc93c03cd8921f   0.0  0.0    0.0  0.0  
4e048c7081dc9d212d105670   0.0  0.0    0.0  0.0  
4e29c9f718a80bb05845f810   0.0  0.0    0.0  0.0  

[5 rows x 1000 columns]

In [22]:
Q.to_csv("Q.csv")

In [33]:
Q = pd.read_csv("Q.csv")

In [36]:
Q = Q.rename(Q.iloc[:, 0])
Q = Q.iloc[:, 1:]

In [38]:
Q.shape

(544, 1000)

Now we are done and words are the inout for the user. This is the entry which asks you whihc type of restaurant you are looking for. This data is tokenised after cleaning and words are analysed and the result obtained is matched with the closed value in the matrix and the venue is recommended. Of all only 3 are returned as most appropriate recommendation. 

In [40]:
words = 'I want to eat best food available here' #Write your input here ....

In [41]:
test_df= pd.DataFrame([words], columns=['Review'])
test_df['Review'] = clean_text(test_df['Review'])
test_vectors = userid_vectorizer.transform(test_df['Review'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Likes'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Likes'],ascending=[0])[:3]

Creating a final_df which will contain all the necessary details of the recommended venue. It has all the columns as regular df but only three entries which are recommended

In [42]:
final_df = pd.DataFrame(columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category',
       'Venue Id', 'Reviews', 'User Id', 'Likes'])
for i in topRecommendations.index.values:
    temp = df.loc[df['Venue Id'] == i]
#     final_df.loc[len(final_df)] = temp
    final_df = final_df.append(temp)

In [64]:
str1 = final_df.iloc[0][3] + " : " + final_df.iloc[0][0] + "                                "
str2 = final_df.iloc[1][3] + " : " + final_df.iloc[1][0]+ "                                "
str3 = final_df.iloc[2][3] + " : " + final_df.iloc[2][0]
ans = '\n'.join([str1, str2, str3])

Final Inspection of the data frame

In [61]:
s = final_df.iloc[0][3] + " : " + final_df.iloc[0][0] + "\n" + final_df.iloc[1][3] + " : " + final_df.iloc[1][0] + "\n" + final_df.iloc[2][3] + " : " + final_df.iloc[2][0] + "\n"

In [65]:
ans

'Shake Shack : New York\nsweetgreen : Boston\nTakahachi : New York'

Plotting the venues in final_df using venue latitudes and columns...

In [193]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lon, poi in zip(final_df['Venue Latitude'], final_df['Venue Longitude'], final_df['Venue']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

This project concludes here. Hope you understood it. It contains a lot of complicated maths but it works !!!
It can easily be extended to multiple locations. It is based on CP, Delhi due to limited API calls. colde will be 99% same and cre tech used is still valid. Hope to see you there at a recommended location.
Thanks for reviewing.